In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import json
import os
from os.path import exists
import time
from rtsvg import *
rt = RACETrack()

In [2]:
_data_ = json.loads(open('../../../data/2024_vast/MC1/mc1.json').read())

In [3]:
_json_repr_ = rt.jsonRepr(_data_)
_df_, _relates_, _labels_ = _json_repr_.starPathGraphDataFrame()
g   = rt.createNetworkXGraph(_df_, _relates_)
pos = rt.hyperTreeLayout(g)
#rt.interactiveGraphPanel(_df_, ln_params={'relationships':_relates_, 'node_labels':_labels_, 'pos':pos}, w=800, h=600)

In [ ]:
_lu_ = {'sort':[]}
for i in range(len(_data_['nodes'])):    
    as_list = list(_data_['nodes'][i].keys())
    as_list.sort()
    _lu_['sort'].append('-'.join(as_list))
rt.co_mgr.optimizeCategoricalColors(set(_lu_['sort']))
svgs = [rt.histogram(pd.DataFrame(_lu_), bin_by='sort', color_by='sort', w=256, h=64)._repr_svg_()]

_lu_ = {'links':[]}
for i in range(len(_data_['links'])):    
    as_list = list(_data_['links'][i].keys())
    as_list.sort()
    _lu_['links'].append('-'.join(as_list))
rt.co_mgr.optimizeCategoricalColors(set(_lu_['links']))
svgs.append(rt.histogram(pd.DataFrame(_lu_), bin_by='links', color_by='links', w=600, h=64)._repr_svg_())

rt.tile(svgs)

In [5]:
ofi = rt.ontologyFrameworkInstance()

#
# nodes
#
for i in range(len(_data_['nodes'])):
    _obj_ = _data_['nodes'][i]
    if   'dob'     in _obj_:
        _country_, _dob_, _id_, _type_ = _obj_['country'], _obj_['dob'], _obj_['id'], _obj_['type']
        _country_uid_ = ofi.resolveUniqIdAndUpdateLookups(_country_, 'obj:Country',  'uniq', 'obj')
        _dob_id_      = ofi.resolveUniqIdAndUpdateLookups(_dob_,     'xsd:date',     'date', 'obj')
        _id_uid_      = ofi.resolveUniqIdAndUpdateLookups(_id_,      _type_,         'uniq', 'sbj,obj')
        ofi.bufferTripleToAddLater(_id_uid_, 'Property.hasCitizenship', _country_uid_)
        ofi.bufferTripleToAddLater(_id_uid_, 'Property.hasBirthDate',   _dob_id_)
    elif 'country' in _obj_:
        _country_, _id_, _type_ = _obj_['country'], _obj_['id'], _obj_['type']
        _country_uid_ = ofi.resolveUniqIdAndUpdateLookups(_country_, 'obj:Country',  'uniq', 'obj')
        _id_uid_      = ofi.resolveUniqIdAndUpdateLookups(_id_,      _type_,         'uniq', 'sbj,obj')
        ofi.bufferTripleToAddLater(_id_uid_, 'Property.incorporatedIn', _country_uid_)
    else:
        _id_, _type_  = _obj_['id'], _obj_['type']
        _id_uid_      = ofi.resolveUniqIdAndUpdateLookups(_id_,      _type_,         'uniq', 'sbj,obj')

#
# links
#
for i in range(len(_data_['links'])):
    _obj_ = _data_['links'][i]
    if '_algorithm' in _obj_:
        _algorithm_, _article_id_, _date_added_ = _obj_['_algorithm'], _obj_['_articleid'], _obj_['_date_added']
        _last_edited_by_, _raw_source_, _key_   = _obj_['_last_edited_by'], _obj_['_raw_source'], _obj_['key']
        _source_, _target_, _type_              = _obj_['source'], _obj_['target'], _obj_['type']
        _algorithm_uid_      = ofi.resolveUniqIdAndUpdateLookups(_algorithm_,      'obj:Algorithm', 'uniq', 'obj')
        _article_id_uid_     = ofi.resolveUniqIdAndUpdateLookups(_article_id_,     'obj:Article',   'uniq', 'obj')
        _date_added_id_      = ofi.resolveUniqIdAndUpdateLookups(_date_added_,     'xsd:date',      'date', 'obj')
        _last_edited_by_uid_ = ofi.resolveUniqIdAndUpdateLookups(_last_edited_by_, 'Entity.Person', 'uniq', 'obj')
        _key_uid_            = ofi.resolveUniqIdAndUpdateLookups(_key_,            'xsd:integer',   'valu', 'obj')
        _source_uid_         = ofi.resolveUniqId(_source_)
        _target_uid_         = ofi.resolveUniqId(_target_)
        link_uid             = ofi.bufferTripleToAddLater(_source_uid_, _type_, _target_uid_)
        ofi.bufferTripleToAddLater(link_uid, 'Source.fromAlgorithm',   _algorithm_uid_)
        ofi.bufferTripleToAddLater(link_uid, 'Source.fromArticle',     _article_id_uid_)
        ofi.bufferTripleToAddLater(link_uid, 'Property.dateAdded',     _date_added_id_)
        ofi.bufferTripleToAddLater(link_uid, 'Property.lastEditedBy',  _last_edited_by_uid_)
        ofi.bufferTripleToAddLater(link_uid, 'Property.hasKey',        _key_uid_)
    else:
        _date_added_                 = _obj_['_date_added']
        _key_                        = _obj_['key']
        _source_, _target_, _type_   = _obj_['source'], _obj_['target'], _obj_['type']
        _date_added_id_  = ofi.resolveUniqIdAndUpdateLookups(_date_added_,  'xsd:date',      'date', 'obj')
        _key_uid_        = ofi.resolveUniqIdAndUpdateLookups(_key_,         'xsd:integer',   'valu', 'obj')
        _source_uid_     = ofi.resolveUniqId(_source_)
        _target_uid_     = ofi.resolveUniqId(_target_)
        link_uid         = ofi.bufferTripleToAddLater(_source_uid_, _type_, _target_uid_)
        ofi.bufferTripleToAddLater(link_uid, 'Property.dateAdded', _date_added_id_)
        ofi.bufferTripleToAddLater(link_uid, 'Property.hasKey',    _key_uid_)

ofi.appendBufferedTriplesAndClearBuffer()

In [ ]:
params = {'w':256, 'h':384, 'color_by':'vrb'}
_df_   = ofi.df_triples
#_df_   = ofi.df_triples.filter(pl.col('stype') != '__triple__')
rt.tile([rt.histogram(_df_, bin_by='sbj',   **params),
         rt.histogram(_df_, bin_by='stype', **params),
         rt.histogram(_df_, bin_by='vrb',   **params),
         rt.histogram(_df_, bin_by='obj',   **params),
         rt.histogram(_df_, bin_by='otype', **params)])

In [ ]:
ofi.to_files('../../../data/2024_vast/MC1/2024_rt_ontology')
#rt.ontologyFrameworkInstance().fm_files('../../../data/2024_vast/MC1/2024_rt_ontology')

In [ ]:
# Issue... the histogram is not properly rendered if there are labels and the color by equals the labeled column...
# ... even worse, the labels mapping could be ambiguous -- does it apply to just bin_by column? or count_by? or color_by?
_df_ = pl.DataFrame({'bin':['a','b']})
rt.tile([rt.histogram(_df_, bin_by='bin', labels={'a':'and','b':'by'}, color_by='bin', h=64),
         rt.histogram(_df_, bin_by='bin', color_by='bin', h=64)])

In [ ]:
_df_orig_    = ofi.df_triples

# Only keep events
vrbs_to_keep = set()
for _vrb_ in set(_df_orig_['vrb']):
    if _vrb_.startswith('Event.'): vrbs_to_keep.add(_vrb_)
_df_ = _df_orig_.filter(pl.col('vrb').is_in(vrbs_to_keep))
print(f'Filtered ... {len(_df_)=}')

# Add in the actual entity names
_df_ = _df_.with_columns(pl.col('sbj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('sbj_str'),
                         pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('obj_str'))
_df_ = _df_.drop(['sdisp', 'odisp', 'grp', 'gdisp', 'src'])

# Add in the edit date
_df_join_ = _df_orig_.filter(pl.col('vrb') == 'Property.dateAdded').with_columns(pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('timestamp'))
_df_join_ = _df_join_.drop(['uid', 'stype', 'sdisp', 'vrb', 'obj', 'otype', 'odisp', 'grp', 'gdisp', 'src'])
_df_      = _df_.join(_df_join_, left_on='uid', right_on='sbj')
print(f'Edit Date Joined ... {len(_df_)=}')

# Add in the last edited by
_df_join_ = _df_orig_.filter(pl.col('vrb') == 'Property.lastEditedBy').with_columns(pl.col('obj').map_elements(lambda x: ofi.uid_lu[x][0], return_dtype=pl.String).alias('last_edited_by'))
_df_join_ = _df_join_.drop(['uid', 'stype', 'sdisp', 'vrb', 'obj', 'otype', 'odisp', 'grp', 'gdisp', 'src'])
_df_      = _df_.join(_df_join_, left_on='uid', right_on='sbj')
print(f'Last Edited By Joined ... {len(_df_)=}')

# Convet the timestamp string to a timestamp
_df_      = rt.columnsAreTimestamps(_df_, 'timestamp')

rt.temporalBarChart(_df_, color_by='vrb')

In [ ]:
rt.smallMultiples(_df_, category_by='last_edited_by', sm_type='temporalBarChart', color_by='vrb', w=1536, w_sm_override=128, h_sm_override=80)

In [ ]:
rt.smallMultiples(_df_, category_by='obj_str', sm_type='temporalBarChart', x_axis_independent=False,
                  color_by='vrb', w=1536, w_sm_override=128, h_sm_override=80)